In [40]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [41]:
dataset = Planetoid("data", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])

In [42]:
data = dataset[0]

In [43]:
data.train_mask = data.test_mask = data.val_mask = None
data

Data(edge_index=[2, 9104], x=[3327, 3703], y=[3327])

In [44]:
data = train_test_split_edges(data)
data

Data(test_neg_edge_index=[2, 455], test_pos_edge_index=[2, 455], train_neg_adj_mask=[3327, 3327], train_pos_edge_index=[2, 7740], val_neg_edge_index=[2, 227], val_pos_edge_index=[2, 227], x=[3327, 3703], y=[3327])

In [45]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2*out_channels, cached=True)
        self.conv2 = GCNConv(2*out_channels, out_channels, cached=True)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

In [46]:
from torch_geometric.nn import GAE

In [47]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
#train_pos_edge_index = data[data.train_pos_edge_index].to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# initialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [48]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [49]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    #print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

In [50]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[-0.5537,  0.0151],
        [ 0.2227,  1.0773],
        [-0.0038, -0.8986],
        ...,
        [-0.3668,  1.0752],
        [-0.5126, -0.4053],
        [-0.3554, -0.9001]], grad_fn=<AddBackward0>)

In [51]:
from torch.utils.tensorboard import SummaryWriter

In [52]:
writer = SummaryWriter('runs/GAE_experiment_'+'2d_100_epochs')

In [53]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    #print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    writer.add_scalar('auc train', auc, epoch)
    writer.add_scalar('ap train', ap, epoch)

# VGAE

In [54]:
from torch_geometric.nn import VGAE

In [55]:
dataset = Planetoid("data", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.test_mask = data.val_mask = None
data = train_test_split_edges(data)

class VariationalGAEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGAEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2*out_channels, cached=True)
        self.conv_mu = GCNConv(2*out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2*out_channels, out_channels, cached=True)
    
    def forward(self, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [56]:
out_channels = 2
num_features = dataset.num_features
epochs = 300

model = VGAE(VariationalGAEncoder(num_features, out_channels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [59]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
        return model.test(z, pos_edge_index, neg_edge_index)

In [58]:
writer = SummaryWriter('runs/VGAE_experiment_2d_100_epochs/')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    

TypeError: forward() takes 2 positional arguments but 3 were given